In [ ]:
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
import os

from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

load_dotenv(override=True)

DEEPSEEK_API = os.getenv("DEEPSEEK_API")

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=DEEPSEEK_API,
    # other params...
)

from langchain.messages import SystemMessage, HumanMessage, AIMessage
system_message = "你叫小智，是一名乐于助人的智能助手。请在对话中保持温和，有耐心的语气。"
system_message = SystemMessage(content=system_message)

messages = [
    system_message
]


agent = create_agent(
    model=model,
    system_prompt="你是一名多才多艺的智能助手，可以调用工具帮助用户解决问题。",
    middleware=[
        SummarizationMiddleware(
            model=model,
            trigger=("messages", 2),
            summary_prompt="请简要总结以下对话内容，保留关键信息，以便后续使用：\n{history}",
        )
    ],
)

print("输入 exit 退出对话")

while True:
    user_input = input("用户：")
    if user_input.lower() == "exit":
        print("对话结束。")
        break

    messages.append(HumanMessage(content=user_input))

    print("小智：", end="", flush=True)
    full_response = ""

    result = agent.invoke({"messages": messages})
    print(result)
    print("\n" + "-"*50)

    messages.append(AIMessage(content=full_response))


输入 exit 退出对话
小智：{'messages': [SystemMessage(content='你叫小智，是一名乐于助人的智能助手。请在对话中保持温和，有耐心的语气。', additional_kwargs={}, response_metadata={}, id='5367fa7b-9558-4544-8a7a-36ef3bced985'), HumanMessage(content='我叫陈明', additional_kwargs={}, response_metadata={}, id='29c851a9-5b26-4f56-982e-dc70d45f0472'), AIMessage(content='陈明你好！很高兴认识你！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 46, 'total_tokens': 60, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 46}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '679ae3ff-a57c-4a85-841f-3c11b5d94b41', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b737e-cb69-7652-9de1-b86a388ad8f7-0', usage_metadata={'input_tokens': 46, 'output_tokens': 14, 'total_tokens': 60, 'input_token